In [1]:
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
from threading import Thread
import dlib
import os
import cv2

#Run this code on the computer which is running the Emotiv Cortex app.
import requests

In [3]:
# calculate the distance between 2 eyes
def e_dist(pA, pB):
	return np.linalg.norm(pA - pB)

def eye_ratio(eye):
	# Calculate the vertical distance between upper and lower lashes
	d_V1 = e_dist(eye[1], eye[5])
	d_V2 = e_dist(eye[2], eye[4])

	# Calculate the horizontal distance between 2 points
	d_H = e_dist(eye[0], eye[3])

	# ratio between vertical and horizontal
	eye_ratio_val = (d_V1 + d_V2) / (2.0 * d_H)

	return eye_ratio_val

# Defines the threshold level for the unique/horizontal ratio. If it's below this level, you're falling asleep
eye_ratio_threshold = 0.25

# Threshold number of continuous frames with eyes closed
max_sleep_frames = 16

# Dem so frame ngu
sleep_frames = 0

# Check to see if there is a warning
alarmed = False


# Initialize eye detection and facial landmark modules
face_detect = cv2.CascadeClassifier('C:/Users/mahmo/anaconda3/Lib/opencv-master/data/haarcascades/haarcascade_frontalface_default.xml')
landmark_detect = dlib.shape_predictor("C:/Users/mahmo/Desktop/Grad/shape_predictor_68_face_landmarks.dat")

# Get the list of landmark clusters for 2 mat
(left_eye_start, left_eye_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(right_eye_start, right_eye_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# Read from camera
vs = VideoStream(src=0).start()
#time.sleep(1.0)



In [4]:
while True:
    
    # Read from camera
    frame = vs.read()

    # Resize to increase processing speed
    frame = imutils.resize(frame, width=450)

    # color to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in pictures
    faces = face_detect.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100), flags=cv2.CASCADE_SCALE_IMAGE)

    # Browse the faces
    for (x, y, w, h) in faces:

        # Create a rectangle around the face
        rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))

        # Identifying landmarks
        landmark = landmark_detect(gray, rect)
        landmark = face_utils.shape_to_np(landmark)

        # Calculate right and left face ratio and average 2 ratio
        leftEye = landmark[left_eye_start:left_eye_end]
        rightEye = landmark[right_eye_start:right_eye_end]

        left_eye_ratio = eye_ratio(leftEye)
        right_eye_ratio = eye_ratio(rightEye)

        eye_avg_ratio = (left_eye_ratio + right_eye_ratio) / 2.0

        # Ve duong bao quanh mat
        left_eye_bound = cv2.convexHull(leftEye)
        right_eye_bound = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [left_eye_bound], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [right_eye_bound], -1, (0, 255, 0), 1)

        # Check if your eyes are wrong
        if eye_avg_ratio < eye_ratio_threshold:
            sleep_frames += 1
            # if the eyes were closed for a sufficient number of
            # frames, then sound the alarm
            if sleep_frames >= max_sleep_frames:
                if not alarmed:
                    alarmed = True
                    url_get = 'http://192.168.225.71:5000/stop' # Use the ip address of the raspberry pi board.
                    res = requests.get(url_get)
                # About the warning text
                cv2.putText(frame, "WAKE UP WAKE UP!!!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)  
        # If you don't close your eyes
        else:
            url_get = 'http://192.168.225.71:5000/go' # Use the ip address of the raspberry pi board.
            res = requests.get(url_get)
            # Reset parameters
            sleep_frames = 0
            alarmed = False
            # Displays average eye ratio value
            cv2.putText(frame, "EYE AVG RATIO: {:.3f}".format(eye_avg_ratio), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)   
    # Display on screen
    cv2.imshow("Camera", frame)
    #time.sleep(1)
    #Press Esc to exit
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cv2.destroyAllWindows()
vs.stop()
